pseudo

    sort points along axis
    
    take first 2 points and create convex hull
    
    for each other point
        find r tangent
        find l tangent
            add all edges between point and points []between r and l
            remove points ][ between r and l and replace with new point


In [170]:
from collections import defaultdict

class triangulate():
    
    def __init__(self,inx):
        # input format
        linx = inx.split('\n')
        self.points = [tuple(int(x) for x in z.split(' ')[:2]) for z in linx[1:]]
        # sort along axis
        self.points.sort(key = lambda x:(x[0],x[1]))  
        self.edges = defaultdict(list)
        # hull
        self.H = self.points[:2] + self.points[:1]
        self.n = len(self.H)
        # possible tangents
        self.T = {'top':{(0,-1),(-1,-1)},'bottom':{(1,1),(1,0)}}
        
    def tang(self,a,b,P):
        # P below,on or above line a-b
        t = (P[0]-b[0])*(a[1]-b[1]) - (a[0]-b[0])*(P[1]-b[1]) 
        if t > 0: return 1
        if not t: return 0
        return -1
       
    def binary_search_hull(self,P,direction):
        sol = self.T[direction]
        # test Hull[0]
        if (self.tang(self.H[0],self.H[-2],P), self.tang(self.H[0],self.H[1],P)) in sol:
            return 0
        
        start = 0
        end = self.n - 1
        while True:
            mid = (start+end)//2
            t_in = self.tang(self.H[mid],self.H[mid-1],P)
            t_out = self.tang(self.H[mid],self.H[mid+1],P)        
            if (t_in,t_out) in sol:
                return mid
            
            next_start = self.tang(self.H[start+1],P,self.H[start])
            start_mid = self.tang(self.H[start],P,self.H[mid]) 
            if direction == 'top':
                if next_start == 1:
                    if t_out == -1 or start_mid == 1: end = mid
                    else: start = mid
                else:
                    if t_out >= 0 or start_mid >= 0: start = mid
                    else: end = mid
            else:
                if next_start == -1:
                    if t_out >= 0 or start_mid == -1: end = mid
                    else: start = mid
                else:
                    if t_out == -1 or start_mid <= 0: start = mid
                    else: end = mid    
                    
    def polygon_tangents(self,P):
        b = self.binary_search_hull(P,'bottom')
        t = self.binary_search_hull(P,'top')
        return b,t

    def grow_hull(self,P):
        # intial n > 2 points colinear
        if len(self.H) == 3: 
            if P[0] == self.H[0][0] == self.H[1][0]:
                self.H[1] = point
                pass 
            else:
                # initial edges
                for x,y in zip(self.H[:2],self.H[:2][::-1]):
                    self.edges[x].append(y)           
        b,t = self.polygon_tangents(P)
        if b < t:
            for x in self.H[b:t+1]:
                self.edges[x].append(P)
                self.edges[P].append(x)
            self.H[b+1:t] = []
        else:
            for x in self.H[b:-1]+self.H[:1]:
                self.edges[x].append(P)
                self.edges[P].append(x)
            self.H[b+1:-1] = []
        self.H.insert(b+1,P)
        self.n = len(self.H)
        pass 

    def solve(self):
        for P in self.points[2:]:
            self.grow_hull(P)
        return self.edges

inx = '8\n0 0 \n0 6 \n6 0 \n6 6 \n2 2 \n2 4 \n4 2 \n4 4'

problem = triangulate(inx)
problem.solve()

defaultdict(list,
            {(0, 0): [(0, 6), (2, 2), (4, 2), (6, 0)],
             (0, 6): [(0, 0), (2, 2), (2, 4), (4, 2), (4, 4), (6, 6)],
             (2, 2): [(0, 6), (0, 0), (2, 4), (4, 2)],
             (2, 4): [(0, 6), (2, 2), (4, 2)],
             (4, 2): [(0, 6), (2, 4), (2, 2), (0, 0), (4, 4), (6, 0)],
             (4, 4): [(0, 6), (4, 2), (6, 0), (6, 6)],
             (6, 0): [(4, 4), (4, 2), (0, 0), (6, 6)],
             (6, 6): [(0, 6), (4, 4), (6, 0)]})

In [171]:
problem = triangulate(inx)

%timeit -n1 -r1 problem.solve()

1 loop, best of 1: 279 µs per loop


In [185]:
import random
from math import floor
L = [' '.join(list(map(str,[floor(random.random()*10000),floor(random.random()*10000)]))) for x in range(1000)]
inx = '1000\n'+'\n'.join(L)

In [186]:
problem = triangulate(inx)

%timeit -n1 -r1 problem.solve()

1 loop, best of 1: 113 ms per loop


In [150]:
problem.H

[(1, 747),
 (11, 984),
 (116, 985),
 (142, 943),
 (159, 337),
 (159, 209),
 (154, 29),
 (88, 7),
 (4, 59),
 (1, 747)]

In [151]:
len(problem.edges)

48

In [153]:
problem.points[48]

(159, 862)